# Tutorial 09: Inflows

This tutorial walks you through the process of introducing inflows of vehicles into a network. Inflows allow us to simulate open networks where vehicles may enter (and potentially exit) the network. This exercise is organized as follows: in section 1 we prepare our inflows variables to support inflows into a merge network supplied by Flow, and in section 2 we simulate the merge network in the presence of inflows.

## 1. Adding Inflows

For this exercise, we will simulate inflows through a highway network with an on-merge. As we will see, the perturbations caused by vehicles entering through the on-merge leads the formation of congested waves downstream in the main highway.

We begin by importing the merge scenario class provided by Flow.

In [1]:
from flow.scenarios.merge import MergeScenario

A schematic of the above network is availabe in the figure below. As we can see, the edges at the start of the main highway and the on-merge are named "inflow_highway" and "inflow_merge" respectively. These names will be important to us when we begin specifying our inflows into the network.

<img src="img/merge_scheme.png" width="750">

We will also define the types of vehicles that are placed in the network. These types of vehicles will also be of significance to us once the inflows are being defined. For this exercise, we add only one type of vehicle to the network, with the vehicle identifier "human":

In [2]:
from flow.core.vehicles import Vehicles
from flow.controllers import IDMController

# create an empty vehicles object
vehicles = Vehicles()

# add some vehicles to this object of type "human"
vehicles.add("human", 
             acceleration_controller=(IDMController, {}),
             speed_mode="no_collide",  # we use the speed mode "no_collide" for better dynamics at the merge
             num_vehicles=20)

Next, we are ready to import and create an empty inflows object.

In [3]:
from flow.core.params import InFlows

inflow = InFlows()

The `InFlows` object is provided as an input during the scenario creation process via the `NetParams` parameter. Introducing these inflows into the network is handled by the backend scenario generation processes during instantiation of the scenario object.

In order to add new inflows of vehicles of pre-defined types onto specific edges and lanes in the network, we use the `InFlows` object's `add` method. This function accepts the following parameters:

* **veh_type**: type of vehicles entering the edge, must match one of the types set in the Vehicles class
* **edge**: starting edge for vehicles in this inflow, must match an edge name in the network
* **veh_per_hour**: number of vehicles entering from the edge per hour, may not be achievable due to congestion and safe driving behavior
* other parameters, including: **start**, **end**, and **probability**. See documentation for more information.

In addition to the above parameters, several optional inputs to the `add` method may be found within sumo's documentation at: http://sumo.dlr.de/wiki/Definition_of_Vehicles,_Vehicle_Types,_and_Routes. Some important features include:

* **departLane**: specifies which lane vehicles will enter from on the edge, may be specified as "all" or "random"
* **departSpeed**: speed of the vehicles once they enter the network

We begin by adding inflows of vehicles at a rate of 2000 veh/hr through *all* lanes on the main highways as follows:

In [4]:
inflow.add(veh_type="human",
           edge="inflow_highway",
           vehs_per_hour=2000,
           departSpeed=10,
           departLane="random")

Next, we specify a second inflow of vehicles through the on-merge lane at a rate of only 100 veh/hr.

In [5]:
inflow.add(veh_type="human",
           edge="inflow_merge",
           vehs_per_hour=100,
           departSpeed=10,
           departLane="random")

## 2. Running Simulations with Inflows

We are now ready to test our inflows in simulation. As mentioned in section 1, the inflows are specified in the `NetParams` object, in addition to all other network-specific parameters. For the merge network, this is done as follows: 

In [6]:
from flow.scenarios.merge import ADDITIONAL_NET_PARAMS
from flow.core.params import NetParams

additional_net_params = ADDITIONAL_NET_PARAMS.copy()

# we choose to make the main highway slightly longer
additional_net_params["pre_merge_length"] = 500

net_params = NetParams(inflows=inflow,  # our inflows
                       no_internal_links=False,
                       additional_params=additional_net_params)

Finally, we execute the simulation following simulation creation techniques we learned from exercise 1 using the below code block. Running this simulation, we see an excessive number of vehicles entering from the main highway, but only a sparse number of vehicles entering from the on-merge. Nevertheless, this volume of merging vehicles is sufficient to form congestive patterns within the main highway.

In [8]:
from flow.core.params import SumoParams, EnvParams, InitialConfig
from flow.envs.loop.loop_accel import AccelEnv, ADDITIONAL_ENV_PARAMS
from flow.core.experiment import SumoExperiment

sumo_params = SumoParams(render=True,
                         sim_step=0.2)

env_params = EnvParams(additional_params=ADDITIONAL_ENV_PARAMS)

initial_config = InitialConfig()

scenario = MergeScenario(name="merge-example",
                         vehicles=vehicles,
                         net_params=net_params,
                         initial_config=initial_config)

env = AccelEnv(env_params, sumo_params, scenario)

exp = SumoExperiment(env, scenario)

_ = exp.run(1, 1500)

**********************************************************
**********************************************************
**********************************************************
significantly decrease after large number of rollouts. In 
order to avoid this, set SumoParams(restart_instance=True).
**********************************************************
**********************************************************
**********************************************************
Round 0, return: 461.69990202420087
Average, std return: 461.69990202420087, 0.0
Average, std speed: 4.068107238565223, 0.0
